In [1]:
import numpy as np 
import pandas as pd
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 37.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompat

In [2]:
train_df = pd.read_csv(r'/kaggle/input/train-csv/train.csv')

In [3]:
test_df = pd.read_csv('/kaggle/input/train-csv/train.csv')

In [4]:
train_df = train_df.dropna()

In [5]:
train_df.iloc[:,0].value_counts()

2
2    1799912
1    1799880
Name: count, dtype: int64

# RNN 

In [6]:
sentences = [review.split() for review in train_df.iloc[:,2]]

In [7]:
from gensim.models import KeyedVectors,Word2Vec

In [8]:
model = Word2Vec(sentences, vector_size=16, window=3, min_count=100000, workers=1000) 
model.wv.save_word2vec_format("got_word2vec.txt", binary=False)

In [9]:
word_vectors = KeyedVectors.load_word2vec_format("got_word2vec.txt", binary=False)

In [10]:
def words_to_vector(text, model, vector_size=16):
    
    words = text.split()  # Tokenize text
    vectors = [model[w] for w in words if w in model]  # Convert words to embeddings
    return np.mean(vectors, axis=0) if vectors else np.zeros(vector_size)

In [11]:
x = [words_to_vector(review,word_vectors,16) for review in train_df.iloc[:,2]]
y = train_df.iloc[:,0]
x = np.array(x)
y = np.array(y)
x = x.reshape((x.shape[0], 1, x.shape[1]))

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_tensor = torch.tensor(x, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)
y_tensor = (y_tensor.view(-1, 1)).float()
y_tensor = y_tensor - 1
import gc
del x, y
gc.collect()

class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,num_classes):
        super(RNN,self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size,hidden_size,num_layers,batch_first = True,nonlinearity='tanh')
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
            )
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)  
        out = out[:, -1, :]  
        out = self.fc(out) 
        return out

input_size = 16
hidden_size = 64
num_layers = 2
num_classes = 2
learning_rate = 0.01

model = RNN(input_size,hidden_size,num_layers,num_classes).to(device)
del RNN,input_size,hidden_size,num_layers,
gc.collect()
criterion = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
y_tensor = y_tensor.view(-1, 1)
for epoch in range(100):
    optimizer.zero_grad()  
    outputs = model(x_tensor) 
    loss = criterion(outputs, y_tensor)  
    loss.backward()  
    optimizer.step() 
    del outputs
    gc.collect()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/100], Loss: {loss.item():.4f}')

In [13]:
del criterion,optimizer,train_df,sentences
gc.collect()

0

In [ ]:
x_test = np.array([words_to_vector(review,word_vectors,16) for review in test_df.iloc[:,2]])
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
del x_test,word_vectors,words_to_vector
predictions = model(x_test_tensor)
del model
gc.collect()

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
y_test = np.array(test_df.iloc[:,0])
y_test = torch.tensor(y_test, dtype=torch.long).to(device)
y_test = (y_test.view(-1, 1)).float()

accuracy = accuracy_score(y_test, predictions.cpu().numpy())
print(f'Accuracy: {accuracy:.4f}')
f1 = f1_score(y_test, predictions.cpu().numpy(), average="binary") 
print(f'F1-Score: {f1:.4f}')
conf_matrix = confusion_matrix(y_test, predictions.cpu().numpy())
print("Confusion Matrix:")
print(conf_matrix)

# LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # Replace RNN with LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer with sigmoid for binary classification
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  
        out = out[:, -1, :]  # Take last output in sequence
        out = self.fc(out)  # Apply classification layer
        return out

# Hyperparameters
input_size = 16
hidden_size = 64
num_layers = 2
num_classes = 2
learning_rate = 0.01

# Initialize model
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Training loop
for epoch in range(100):
    optimizer.zero_grad()  
    outputs = model(x_tensor)  # Remove `torch.no_grad()` (needed for training)
    loss = criterion(outputs, y_tensor)  
    loss.backward()  
    optimizer.step() 

    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/100], Loss: {loss.item():.4f}')

# Clean up memory
del x_tensor, y_tensor
gc.collect()

In [ ]:
predictions = model(x_test_tensor)

In [ ]:
accuracy = accuracy_score(y_test, predictions.cpu().numpy())
print(f'Accuracy: {accuracy:.4f}')
f1 = f1_score(y_test, predictions.cpu().numpy(), average="binary") 
print(f'F1-Score: {f1:.4f}')
conf_matrix = confusion_matrix(y_test, predictions.cpu().numpy())
print("Confusion Matrix:")
print(conf_matrix)